# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("../output_data/cities.csv")
city_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mackay,-21.15,149.20,86.00,58,25,14.99,AU,1603856694
1,Hobart,-42.88,147.33,62.01,82,75,16.11,AU,1603856514
2,Rikitea,-23.12,-134.97,75.42,84,100,15.50,PF,1603856700
3,Vaini,-21.20,-175.20,78.80,74,20,9.17,TO,1603857147
4,Nikolskoye,59.70,30.79,52.00,87,90,11.18,RU,1603857148


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key=g_key)
location = city_data[['Lat', 'Lng']]
weights = city_data['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(location, weights=weights, max_intensity=100, point_radius=20))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
fit_city_data = city_data.drop(city_data[ (city_data['Max Temp'] < 65)].index, inplace=False)
fit_city_data = fit_city_data.drop(fit_city_data[ (fit_city_data['Max Temp'] > 85)].index, inplace=False)
fit_city_data = fit_city_data.drop(fit_city_data[ fit_city_data['Cloudiness'] > 20].index, inplace=False)
fit_city_data = fit_city_data.drop(fit_city_data[ fit_city_data['Humidity'] > 50].index, inplace=False)
fit_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,Busselton,-33.65,115.33,75.99,32,14,7.83,AU,1603856692
54,Tari,12.41,4.49,73.56,21,0,5.57,NG,1603857207
62,Upington,-28.45,21.26,66.20,32,0,2.24,ZA,1603857217
72,Morwa,22.90,73.83,80.56,34,0,6.82,IN,1603857229
75,Burhānpur,21.30,76.23,80.44,41,0,3.85,IN,1603857233
108,Geraldton,-28.77,114.60,75.20,47,0,19.46,AU,1603856706
220,Ginda,30.69,78.49,66.67,26,0,3.15,IN,1603857413
226,Mongo,12.18,18.69,75.79,26,0,4.21,TD,1603857419
228,Taoudenni,22.68,-3.98,74.46,21,0,1.97,ML,1603857421
245,La Reforma,25.08,-108.05,72.00,40,0,13.00,MX,1603857445


In [17]:
len(fit_city_data)

27

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_names = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index,row in fit_city_data.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    target_radius = 5000
    target_type = "lodging"

    params = {"location": str(f'{lat}, {lng}'),
              "radius": target_radius,
              "type": target_type,
              "key": g_key}

    response = requests.get(base_url, params=params).json()
    try:
        hotel_names.append(response["results"][0]["name"])
    except:
        hotel_names.append('No Hotel Found')

In [19]:
hotel_names

['Observatory Guest House',
 'No Hotel Found',
 'River Place Manor',
 'Morva',
 'Vastushilp Guest House',
 'Broadwater Mariner Resort',
 'Bal uma sankar hotel and restaurant',
 'No Hotel Found',
 'No Hotel Found',
 'Arremar Angostura',
 'No Hotel Found',
 'Al Madina Hotal',
 'Hôtel Amadou Seck',
 'ibis Styles Karratha',
 'Siang Guest House',
 'No Hotel Found',
 'Harding River Caravan Park',
 'Duke Hotel & Restaurant',
 'Country Lodge Mongu',
 'No Hotel Found',
 'Gerasimos Safari Lodge',
 'Hotel Green Park',
 'Hôtel Telwa Bungalow',
 'Lovelysweets',
 'Samrat hotel and aman Mobile Showroom',
 'No Hotel Found',
 'No Hotel Found']

In [20]:
fit_city_data['Hotel Name'] = hotel_names

In [21]:
fit_city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,Busselton,-33.65,115.33,75.99,32,14,7.83,AU,1603856692,Observatory Guest House
54,Tari,12.41,4.49,73.56,21,0,5.57,NG,1603857207,No Hotel Found
62,Upington,-28.45,21.26,66.20,32,0,2.24,ZA,1603857217,River Place Manor
72,Morwa,22.90,73.83,80.56,34,0,6.82,IN,1603857229,Morva
75,Burhānpur,21.30,76.23,80.44,41,0,3.85,IN,1603857233,Vastushilp Guest House
108,Geraldton,-28.77,114.60,75.20,47,0,19.46,AU,1603856706,Broadwater Mariner Resort
220,Ginda,30.69,78.49,66.67,26,0,3.15,IN,1603857413,Bal uma sankar hotel and restaurant
226,Mongo,12.18,18.69,75.79,26,0,4.21,TD,1603857419,No Hotel Found
228,Taoudenni,22.68,-3.98,74.46,21,0,1.97,ML,1603857421,No Hotel Found
245,La Reforma,25.08,-108.05,72.00,40,0,13.00,MX,1603857445,Arremar Angostura


In [22]:
hotel_df = fit_city_data.drop(fit_city_data[(fit_city_data['Hotel Name'] == 'No Hotel Found')].index, inplace = False)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,Busselton,-33.65,115.33,75.99,32,14,7.83,AU,1603856692,Observatory Guest House
62,Upington,-28.45,21.26,66.20,32,0,2.24,ZA,1603857217,River Place Manor
72,Morwa,22.90,73.83,80.56,34,0,6.82,IN,1603857229,Morva
75,Burhānpur,21.30,76.23,80.44,41,0,3.85,IN,1603857233,Vastushilp Guest House
108,Geraldton,-28.77,114.60,75.20,47,0,19.46,AU,1603856706,Broadwater Mariner Resort
220,Ginda,30.69,78.49,66.67,26,0,3.15,IN,1603857413,Bal uma sankar hotel and restaurant
245,La Reforma,25.08,-108.05,72.00,40,0,13.00,MX,1603857445,Arremar Angostura
293,Uthal,25.81,66.62,79.66,39,0,0.60,PK,1603857242,Al Madina Hotal
308,Linguère,15.40,-15.12,76.62,18,0,4.85,SN,1603857518,Hôtel Amadou Seck
349,Karratha,-20.74,116.85,83.23,50,5,12.71,AU,1603857566,ibis Styles Karratha


In [24]:
len(hotel_df)

19

In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))